# Calculate utilities of two job decisions.


In [42]:
import numpyro
import numpyro.distributions as dist
from jax import random
import plotly.express as px
import numpy as np

In [2]:
rng_key = random.PRNGKey(0)

In [30]:
# https://realpython.com/python-super/#an-overview-of-pythons-super-function
class Pert(dist.Beta):
    """Class for modified-PERT distribution
    with parameters
    a: min
    b: mode
    c: max
    gamma: Lower values of gamma make for a
        distribution that is less peaked 
        at the mode.
    """
    def __init__(self, a, b, c, gamma=4):
        # https://pubsonline.informs.org/doi/epdf/10.1287/ited.1080.0013
        # Davis 2008 formula for conversion
        # between PERT and beta distributions
        # https://en.wikipedia.org/wiki/PERT_distribution#The_modified-PERT_distribution
        mu = (a + gamma * b + c)/(gamma + 2)
        # https://reference.wolfram.com/language/ref/PERTDistribution.html
        sigma_squared = (c - a - b*gamma + c*gamma)*(c + b*gamma - a * (1 + gamma))/((2 + gamma)**2 * (3 + gamma))
        alpha_plus_beta = (mu - a)*(c - mu)/sigma_squared - 1 
        alpha = (mu - a)/(c - a)*alpha_plus_beta
        beta = (c - mu)/(c - a)*alpha_plus_beta
        concentration1 = alpha
        concentration0 = beta
        super().__init__(concentration1, concentration0)

In [59]:
a = 69.5
b = 74
c = 76
gamma = 1000
pert = Pert(a, b, c, gamma)
# https://num.pyro.ai/en/0.7.1/getting_started.html
y = a + pert.sample(key=rng_key, sample_shape=(100_000,))*(c - a)

In [60]:
fig = px.histogram(y)
fig.show()

In [49]:
np.mean(y)

Array(72.993835, dtype=float32)

In [50]:
(a + b*gamma + c)/(2 + gamma)

73.0

In [51]:
np.var(y)

Array(3.0123384, dtype=float32)

In [52]:
(c - a - b*gamma + c*gamma)*(c + b*gamma - a * (1 + gamma))/((2 + gamma)**2 * (3 + gamma))

3.0